# Stolen Bases: A Matter of Skill or Guts?

<img src='https://img.mlbstatic.com/mlb-images/image/upload/t_2x1/t_w1536/mlb/yblyorebwvue0kwl7y0b.jpg' width='600' align='center'/>

# Business Understanding

MLB saw an increase in stolen bases in 2023. An MLB team wants to increase viewer retention rates and increase the fan base by adding more action into the game, and they've decided on joining the fun with stolen bases. In this project, I will be advising the MLB team on **how to increase SB stats** for their players. Is it a matter of skill or guts? While this MLB team wants to increase SB stats, they do not want it to jeopardize their wins.

I investigate the following questions:
1. Was the increase in stolen bases from 2022 to 2023 significant?
2. What is the number of **stolen base attempts** that would **increase action** in the game ***without*** greatly **increasing the chances of a loss**?

# Data Understanding

This data was extracted from a custom leaderboard I created on **Baseball Savant**. There are 2 datasets with the same format - [one from 2022](https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=batter&filter=&min=q&selections=r_total_caught_stealing%2Cr_total_stolen_base%2Cn_bolts%2Csprint_speed&chart=true&x=r_total_caught_stealing&y=r_total_caught_stealing&r=no&chartType=beeswarm&sort=r_total_stolen_base&sortDir=desc) and [one from 2023](https://baseballsavant.mlb.com/leaderboard/custom?year=2023&type=batter&filter=&min=q&selections=r_total_caught_stealing%2Cr_total_stolen_base%2Cn_bolts%2Csprint_speed&chart=true&x=r_total_caught_stealing&y=r_total_caught_stealing&r=no&chartType=beeswarm&sort=r_total_stolen_base&sortDir=desc). The data includes **CS** (caught stealing), **SB** (stolen base), **Bolts** and **Sprint Speed** from **players in the MLB** (Major League Baseball). The 2022 dataset has stats for 130 players and the 2023 dataset has stats for 133 players. This data has been collected from **Statcast**, "a state-of-the-art tracking technology, capable of measuring previously unquantifiable aspects of the game."([Baseball Savant](https://baseballsavant.mlb.com/about#:~:text=Where%20is%20the%20data%20from,probable%20pitchers%20for%20upcoming%20days.))

I began with importing the necessary libraries for my data preparation and exploratory data analysis. These libraries are for data manipulation and data visualization.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

I then imported the data as pandas dataframes. The 2022 data is saved as `df_2022` and the 2023 data is saved as `df_2023`.

In [15]:
# import datasets
df_2022 = pd.read_csv('stats_2022.csv')
df_2023 = pd.read_csv('stats_2023.csv')

## Data Preparation

Since the data was extracted from a custom leaderboard I created on [Baseball Savant](https://baseballsavant.mlb.com/), there was minimal cleaning that needed to be done. Both tables follow the same format so I will be following the same cleaning steps for each.

First, I previewed the first 5 entries of each table. This gives me a general idea on how I want to clean and handle the data. I notice:
1. I want to **clean the column names**.
2. I already see **NaNs for n_bolts**. I will have to decide how to handle that.
3. Team names are not included in the dataset. I could add team rosters to a variable and feature engineer a column for teams, but I am more **focused on specific players** stealing bases, not on overall team statistics.
4. On Baseball Savant, my customer leaderboard was **sorted** by descending stolen bases but the data did *not* import that way. I will keep that in mind as I proceed with my analysis.

In [16]:
# View the first 5 entries of 2022 data
df_2022.head()

,"last_name, first_name",player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
0,"Thomas, Lane",657041,2022,4,8,39.0,29.4
1,"Canha, Mark",592192,2022,1,3,NaN,27.5
2,"Torres, Gleyber",650402,2022,5,10,NaN,26.9
3,"Acuña Jr., Ronald",660670,2022,11,29,25.0,28.5
4,"Mullins II, Cedric",656775,2022,10,34,18.0,28.5


In [17]:
# View the first 5 entries of 2023 data
df_2023.head()

,"last_name, first_name",player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
0,"Candelario, Jeimer",600869,2023,1,8,NaN,27.5
1,"McMahon, Ryan",641857,2023,5,5,NaN,25.8
2,"Muncy, Max",571970,2023,2,1,NaN,26.9
3,"Soler, Jorge",624585,2023,0,1,NaN,26.6
4,"Edman, Tommy",669242,2023,4,27,7.0,28.8


In [18]:
# View the overall shape, dtypes and null counts for each column in df_2022
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   last_name, first_name    130 non-null    object 
 1   player_id                130 non-null    int64  
 2   year                     130 non-null    int64  
 3   r_total_caught_stealing  130 non-null    int64  
 4   r_total_stolen_base      130 non-null    int64  
 5   n_bolts                  44 non-null     float64
 6   sprint_speed             130 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 7.2+ KB


In [19]:
# View the overall shape, dtypes and null counts for each column in df_2023
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   last_name, first_name    133 non-null    object 
 1   player_id                133 non-null    int64  
 2   year                     133 non-null    int64  
 3   r_total_caught_stealing  133 non-null    int64  
 4   r_total_stolen_base      133 non-null    int64  
 5   n_bolts                  50 non-null     float64
 6   sprint_speed             133 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 7.4+ KB


Above I noticed as I viewed the overall makeup of the data that n_bolts is the only column with **nulls** in either dataframe. I want to handle these so I will investigate further.

In [20]:
# View random 20 entries where n_bolts is NaN
df_2023[df_2023['n_bolts'].isna()].sample(20)

,"last_name, first_name",player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
99,"Santander, Anthony",623993,2023,1,5,NaN,26.7
100,"Bichette, Bo",666182,2023,3,5,NaN,27.1
89,"Hayes, Ke'Bryan",663647,2023,6,10,NaN,27.1
104,"Riley, Austin",663586,2023,1,3,NaN,28.1
117,"Contreras, William",661388,2023,1,6,NaN,26.5
14,"Perez, Salvador",521692,2023,0,0,NaN,24.3
130,"Meneses, Joey",608841,2023,0,0,NaN,25.6
24,"Arenado, Nolan",571448,2023,3,3,NaN,24.9
108,"Renfroe, Hunter",592669,2023,0,0,NaN,26.8
54,"Reynolds, Bryan",668804,2023,1,12,NaN,27.8


"A Bolt is any run where the Sprint Speed (defined as "feet per second in a player's fastest one-second window") of the runner is at least 30 ft/sec."([MLB](https://www.mlb.com/glossary/statcast/bolt)). All of the samples where Bolt is NaN include Sprint Speeds that are less than 30. Therefore, it seems safe to say that these players did not have any bolts. I will **change the NaNs to 0's**.

In [21]:
# Replace NaNs in N_Bolts with 0
df_2022['n_bolts'] = df_2022['n_bolts'].fillna(0)
df_2023['n_bolts'] = df_2023['n_bolts'].fillna(0)

Below shows the **summary statistics** of each dataset. Although all of the data below is numerical, player_id and year are categorical data, so those can be ignored. r_total_caught_stealing, r_total_stolen_base, n_bolts, and sprint_speed, however, give me a general overview of the values I expect to see in the data set.

I do notice the difference in r_total_stolen_base means from 2022 to 2023. The 2022 mean is 8.9 and the 2023 mean is 11.75. It is a difference of 2.85, but **is that significant?** I will analyze further in the EDA. 

In [24]:
# View summary statistics of df_2022.
df_2022.describe()

,player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
count,130.000000,130.0,130.000000,130.000000,130.000000,130.000000
mean,607829.930769,2022.0,2.638462,8.907692,7.338462,27.280769
std,61551.798138,0.0,2.572978,8.823371,20.286916,1.290820
min,443558.000000,2022.0,0.000000,0.000000,0.000000,24.000000
25%,571987.750000,2022.0,1.000000,2.000000,0.000000,26.400000
50%,622163.500000,2022.0,2.000000,6.000000,0.000000,27.350000
75%,663574.000000,2022.0,3.750000,13.750000,1.750000,28.200000
max,683734.000000,2022.0,12.000000,35.000000,131.000000,30.300000


In [23]:
# View summary statistics of df_2023.
df_2023.describe()

,player_id,year,r_total_caught_stealing,r_total_stolen_base,n_bolts,sprint_speed
count,133.000000,133.0,133.000000,133.000000,133.000000,133.000000
mean,633932.578947,2023.0,2.796992,11.751880,7.541353,27.395489
std,49936.834584,0.0,2.859750,12.585108,22.734139,1.242301
min,457759.000000,2023.0,0.000000,0.000000,0.000000,24.300000
25%,605204.000000,2023.0,1.000000,3.000000,0.000000,26.400000
50%,656305.000000,2023.0,2.000000,8.000000,0.000000,27.400000
75%,666969.000000,2023.0,4.000000,16.000000,2.000000,28.300000
max,807799.000000,2023.0,15.000000,73.000000,149.000000,30.500000


Below I checked for **duplicates**. I didn't think there would be any but you never know!

In [25]:
# Check for duplicated
len(df_2022[df_2022.duplicated()])

0

In [26]:
# Check for duplicated
len(df_2023[df_2023.duplicated()])

0

In [27]:
# Rewrite column names in title case for preference
df_2022.columns = df_2022.columns.map(lambda x: x.title())
df_2023.columns = df_2023.columns.map(lambda x: x.title())

In [30]:
# Preview how the df looks now
df_2023.head()

,"Last_Name, First_Name",Player_Id,Year,R_Total_Caught_Stealing,R_Total_Stolen_Base,N_Bolts,Sprint_Speed
0,"Candelario, Jeimer",600869,2023,1,8,0.0,27.5
1,"McMahon, Ryan",641857,2023,5,5,0.0,25.8
2,"Muncy, Max",571970,2023,2,1,0.0,26.9
3,"Soler, Jorge",624585,2023,0,1,0.0,26.6
4,"Edman, Tommy",669242,2023,4,27,7.0,28.8


# Exploratory Data Analysis

There are two things I am looking to explore in this Exploratory Data Analysis:
1. Was the increase in stolen bases from 2022 to 2023 significant?
2. What is the number of stolen base attempts that would increase action in the game without greatly increasing the chances of a loss?

## 1. Was the increase in stolen bases from 2022 to 2023 significant?

In order to determine if the increase in stolen bases from 2022 to 2023 was significant, I am going to conduct a **two sample t-test**. This will determine if there is a significant difference between the stolen base means from 2022 to 2023, as was reported in [articles](https://www.mlb.com/news/mlb-records-3000th-stolen-base-in-2023) and fans alike.

These are the steps I will take to conduct the hypothesis test:
1. Set up null and alternative hypotheses.
2. Choose a significance level.
3. Calculate the test statistic.
4. Determine the critical or p-value).
5. Compare t-value with critical t-value to reject or fail to reject the null hypothesis.

## 2. What is the number of stolen base attempts that would increase action in the game without greatly increasing the chances of a loss?

In order to predict the advantageous number of stolen base attemps that would increase the action in the game without greatly increasing the chances of loss, I am going to analyze the data from `df_2023`.

# Conclusions

## Limitations

## Recommendations

## Next Steps